In [1]:
import os
os.chdir('../')
%pwd

'/Net/Groups/BGI/scratch/ppandey/Side_Quest/Text_Summarization_HF'

STEPS TO FOLLOW: 
- config.yaml
- ~~params.yaml~~
- entity
- configuration manager in src config
- components
- pipeline
- main.py
- app.py

In [3]:
# Start with the ENTITY file
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer: Path

In [5]:
# Write the ConfigurationManager
from txtsummarizer.constants import *
from txtsummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILEPATH,
        params_filepath = PARAMS_FILEPATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create artifacts folder
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        # Create directory
        create_directories([config.root_dir])
        data_tranformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer=config.tokenizer
        )

        return data_tranformation_config

In [8]:
# Write the components file
import os

from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

from txtsummarizer.logging import logger

class DataTransformation:
    def __init__(
        self,
        config: DataTransformationConfig
    ):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer)

    def tokenize_and_prepare_data(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        # switch the tokenizer mode to target tokenization.
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'], # indicates to the model which tokens are real and which are padding
            'labels': target_encodings['input_ids']
        }
    
    def preprocess_and_save_data(self):
        samsum_dataset = load_from_disk(self.config.data_path)
        samsum_dataset_pt = samsum_dataset.map(self.tokenize_and_prepare_data, batched=True)
        # Save data
        samsum_dataset_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_data'))

In [11]:
# Write PIPEPLINE
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.preprocess_and_save_data()
except Exception as e:
    raise e

[2024-04-28 19:49:57,504: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-28 19:49:57,505: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-28 19:49:57,506: INFO: common: created directory at: artifacts]
[2024-04-28 19:49:57,506: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Net/Groups/BGI/scratch/ppandey/miniconda3/envs/txtS/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 174673.69 examples/s]
